In [1]:
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="0, 1, 2, 4"

In [2]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask import dataframe as dd
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import log_loss
import optuna
import gc
xgb.__version__

'1.6.2'

In [3]:
!nvidia-smi

Tue Jan 10 10:49:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA TITAN RTX    On   | 00000000:09:00.0 Off |                  N/A |
| 41%   35C    P8    32W / 280W |    905MiB / 24220MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA TITAN RTX    On   | 00000000:41:00.0  On |                  N/A |
| 41%   

In [4]:
!nvidia-smi -L

GPU 0: NVIDIA TITAN RTX (UUID: GPU-0eb32f58-b8d5-17c0-e952-f4ec26a9353f)
GPU 1: NVIDIA TITAN RTX (UUID: GPU-50aeb092-88f5-4e0b-7f73-32741666f319)


In [5]:
train_x = dd.read_csv('../input/otto_ii/xgtrain_aug_5.csv')
test_x = dd.read_csv('../input/otto_ii/xgval.csv')

In [6]:
train_y = train_x['target']
train_x = train_x[train_x.columns.difference(['target'])]

test_y = test_x['target']
test_x = test_x[test_x.columns.difference(['target'])]

In [7]:
cluster = LocalCUDACluster(n_workers=2)
client = Client(cluster)

2023-01-10 10:49:50,395 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-01-10 10:49:50,395 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-01-10 10:49:50,449 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-01-10 10:49:50,449 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


In [8]:
dtrain = xgb.dask.DaskDMatrix(client, train_x, train_y)

In [9]:
dtest = xgb.dask.DaskDMatrix(client, test_x, test_y)


In [10]:
num_round = 1000

In [11]:
def objective(trial):
        
    params = {
        'objective': trial.suggest_categorical('objective',['multi:softprob']),
        'tree_method': trial.suggest_categorical('tree_method',['gpu_hist']),  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001,0.1),
        #'n_estimators': trial.suggest_categorical('n_estimators', [1000]),
        'max_depth': trial.suggest_int('max_depth', 2, 25),
        #'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
        'eval_metric': trial.suggest_categorical('eval_metric',['logloss']),
        'num_class':9,

    }

    output = xgb.dask.train(client, params, dtrain, num_round)
    
    booster = output['booster']  # booster is the trained model
    booster.set_param({'predictor': 'gpu_predictor'})

    predictions = xgb.dask.predict(client, booster, dtest)
    
    predictions = predictions.compute()

    lloss = log_loss(test_y, predictions)
    
    return lloss

In [12]:
study = optuna.create_study(direction='minimize')

[I 2023-01-10 10:49:54,326] A new study created in memory with name: no-name-b4ea6518-2936-495d-aebf-06861c69ded2


In [13]:
%%time
study.optimize(objective, n_trials=2)

/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[10:49:55] task [xgboost.dask-0]:tcp://127.0.0.1:44335 got new rank 0
[I 2023-01-10 10:59:34,892] Trial 0 finished with value: 0.4715039241537287 and parameters: {'objective': 'multi:softprob', 'tree_method': 'gpu_hist', 'lambda': 1.425155735569395, 'alpha': 0.004060752015672895, 'colsample_bytree': 0.7462412149166504, 'subsample': 0.4824901152281203, 'learning_rate': 0.016063708852258444, 'max_depth': 22, 'min_child_weight': 43, 'eval_metric': 'logloss'}. Best is trial 0 with value: 0.4715039241537287.
/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[10:59:34] task [xgboost.dask-0]:tcp://127.0.0.1:44335 got new rank 0
[I 2023-01-10 10:59:55,981] Trial 1 finished with value: 1.20035841

CPU times: user 19.1 s, sys: 5.6 s, total: 24.7 s
Wall time: 10min


In [14]:
%%time
study.optimize(objective, n_trials=100)

/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[10:59:56] task [xgboost.dask-0]:tcp://127.0.0.1:44335 got new rank 0
[I 2023-01-10 11:00:11,837] Trial 2 finished with value: 0.6685043184700257 and parameters: {'objective': 'multi:softprob', 'tree_method': 'gpu_hist', 'lambda': 7.546601768165152, 'alpha': 2.038910432987714, 'colsample_bytree': 0.5375902533324195, 'subsample': 0.8873803735543141, 'learning_rate': 0.0101543786872048, 'max_depth': 3, 'min_child_weight': 175, 'eval_metric': 'logloss'}. Best is trial 0 with value: 0.4715039241537287.
/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[11:00:11] task [xgboost.dask-0]:tcp://127.0.0.1:44335 got new rank 0
[I 2023-01-10 11:02:45,565] Trial 3 finished with value: 0.4698572687752

CPU times: user 13min 52s, sys: 5min 58s, total: 19min 50s
Wall time: 7h 50min 13s


In [15]:
study.best_trial.params

{'objective': 'multi:softprob',
 'tree_method': 'gpu_hist',
 'lambda': 0.0010575876061231088,
 'alpha': 0.33478631384699725,
 'colsample_bytree': 0.4394546378876128,
 'subsample': 0.8255796441357609,
 'learning_rate': 0.020473158943286263,
 'max_depth': 19,
 'min_child_weight': 25,
 'eval_metric': 'logloss'}

In [16]:
real_test_x = dd.read_csv('../input/otto_ii/xgtest.csv')

In [17]:
real_test_y = real_test_x['target']
real_test_x = real_test_x[real_test_x.columns.difference(['target'])]

In [18]:
drealtest = xgb.dask.DaskDMatrix(client, real_test_x, real_test_y)

In [19]:
params = study.best_trial.params
params['num_class']= 9

output = xgb.dask.train(client, params, dtrain, num_round)
    
booster = output['booster']  # booster is the trained model
booster.set_param({'predictor': 'gpu_predictor'})


predictions = xgb.dask.predict(client, booster, drealtest)
    
predictions = predictions.compute()

log_loss(real_test_y, predictions)

/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[18:50:09] task [xgboost.dask-0]:tcp://127.0.0.1:44335 got new rank 0


0.4430504632299437

In [20]:
%%time
params = study.best_trial.params
params['seed'] = 2022
params['num_class']= 9

output = xgb.dask.train(client, params, dtrain, num_round)
    
booster = output['booster']  # booster is the trained model
booster.set_param({'predictor': 'gpu_predictor'})

predictions = xgb.dask.predict(client, booster, drealtest)
    
predictions = predictions.compute()

log_loss(real_test_y, predictions)

/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[18:53:40] task [xgboost.dask-0]:tcp://127.0.0.1:44335 got new rank 0


CPU times: user 6.87 s, sys: 2.33 s, total: 9.2 s
Wall time: 3min 30s


0.4435049703227145

In [21]:
%%time 
scores = []

for jj in range(15):
    print(jj)
    params = study.best_trial.params
    params['num_class']= 9
    params['seed'] = 5*jj**3 + 137

    output = xgb.dask.train(client, params, dtrain, num_round)

    booster = output['booster']  # booster is the trained model
    booster.set_param({'predictor': 'gpu_predictor'})

    predictions = xgb.dask.predict(client, booster, drealtest)

    predictions = predictions.compute()
    
    error = log_loss(real_test_y, predictions)

    scores.append(error)
    print(error)

0


/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[18:57:10] task [xgboost.dask-0]:tcp://127.0.0.1:44335 got new rank 0


0.4429026214542405
1


/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[19:00:42] task [xgboost.dask-0]:tcp://127.0.0.1:44335 got new rank 0


0.44310151038980744
2


...

0.44288707237441916
13


/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[19:42:50] task [xgboost.dask-0]:tcp://127.0.0.1:44335 got new rank 0


0.44249761943384397
14


/home/tunguz/.local/lib/python3.8/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
[19:46:22] task [xgboost.dask-0]:tcp://127.0.0.1:44335 got new rank 0


0.44321041220501883
CPU times: user 1min 37s, sys: 41.5 s, total: 2min 19s
Wall time: 52min 43s


In [22]:
scores

[0.4429026214542405,
 0.44310151038980744,
 0.44230661762606355,
 0.4423600089363706,
 0.44277320056550507,
 0.4430489046320831,
 0.442724870259187,
 0.4438044021812578,
 0.44340863200081954,
 0.4424844386888307,
 0.4434619500164981,
 0.44286124892959583,
 0.44288707237441916,
 0.44249761943384397,
 0.44321041220501883]

In [23]:
import numpy as np
np.mean(scores)

0.4429222339795694